In [ ]:
%cd /content/drive/My\ Drive/enq_analysis
! git pull
! git add -f feature_extract.ipynb 
! git commit -m ""
! git push


/content/drive/My Drive/enq_analysis
Already up to date.
Aborting commit due to empty commit message.
Everything up-to-date


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install mecab-python3==0.996.5 transformers==3.4.0 fugashi ipadic tensorboardX 

     |████████████████████████████████| 17.1MB 206kB/s 
     |████████████████████████████████| 1.3MB 49.6MB/s 
     |████████████████████████████████| 481kB 47.7MB/s 
     |████████████████████████████████| 13.4MB 253kB/s 
     |████████████████████████████████| 317kB 55.3MB/s 
     |████████████████████████████████| 1.1MB 50.5MB/s 
     |████████████████████████████████| 2.9MB 39.4MB/s 
     |████████████████████████████████| 890kB 31.5MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-cp36-none-any.whl size=13556725 sha256=eff22e51608e75e0f9d752063c980afa190ffaef4a92cf4ced0cc3770a795185
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8eacee15cc571ee97fcfc6ef0c8d83dfa1bc8df419bc9bb9dd7bbde7bfd612b0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built ipad

In [6]:
import pandas as pd
from sklearn import datasets
import MeCab
tagger = MeCab.Tagger("-Owakati")
tagger.parse('')
import scipy 
import numpy
import math
import json
import collections
import pathlib
import os
import torch
from transformers.file_utils import is_torch_tpu_available
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers.modeling_bert import BertModel
import unicodedata

In [7]:
if is_torch_tpu_available():
  print("We will use TPU")
  import torch_xla.core.xla_model as xm
  device = xm.xla_device()
elif torch.cuda.is_available():
  print("We will use GPU: %s" % torch.cuda.get_device_name(0))
  device = torch.device('cuda:0')
else:
  print("No GPU available, use CPU instead.")
  device = torch.device('cpu')

We will use GPU: Tesla T4


In [8]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

model.to(device)
model.eval()
model.device

device(type='cuda', index=0)

In [13]:
def sentence_representation(text):
  encoded = tokenizer.encode_plus(preprocess_text(text), return_tensors='pt',max_length=512,truncation=True,).to(device)
  encoded["return_dict"] = True
  encoded["output_hidden_states"] = True
  with torch.no_grad():
    result = model(**encoded)
    return torch.mean(result.hidden_states[-2], 1)[0]

def preprocess_text(text):
	# 先頭・末尾の空白を削除
	text = text.strip()
  # NFKCで文字の正規化
	text = unicodedata.normalize("NFKC", text)
	return text

def sim(text1, text2):
  vec1 = sentence_representation(text1)
  vec2 = sentence_representation(text2)
  return torch.dot(vec1, vec2) / (torch.norm(vec1) * torch.norm(vec2))

def isStopWord(base):
  if base in stopwords:
    return True
  if len(base) == 1:
    return True
  return False

def isContentWord(node):
  features = node.feature.split(',')
  pos1 = features[0]
  pos2 = features[1]
  base = get_base(node)
  #print("surface: "+node.surface+", pos1: "+pos1+", pos2: "+pos2)
  if pos1 == "名詞" or pos1 == "動詞" or pos1 == "形容詞" or pos1 == "形容動詞":
    if pos2 != "非自立" and pos2 != "代名詞" and pos2 != "接尾":
      #print("content word!!: "+node.surface)
      return True
  return False

def isSymbol(node):
  features = node.feature.split(',')
  pos1 = features[0]
  return (pos1 == "記号" or pos1 == "補助記号" or get_base(node) == "*")

def get_base(node):
  tokens = node.feature.split(',')
  if len(tokens) > 10: 
    return tokens[10]
  else:
    return node.surface


def inc(dic, key):
  if key in dic.keys():
    dic[key] += 1
  else:
    dic[key] = 0

def get_ngrams(sentence):
  ngrams = []
  prev_tri = ""
  prev_bi = ""
  prev_uni = ""
  prev_node = None
  prev2_node = None
  prev3_node = None
  node = tagger.parseToNode(sentence)
  while node:
    unigram = node.surface
    pos = node.feature.split(',')[0]
    base = get_base(node)
    quadgram = prev_tri + base
    if prev3_node and not isSymbol(prev3_node) and not isSymbol(node):
      ngrams.append(quadgram)
    trigram = prev_bi + base
    #if prev2_node and isContentWord(prev2_node) and not isSymbol(node):
    if prev2_node and not isSymbol(prev2_node) and not isSymbol(node):
      #print(trigram)
      ngrams.append(trigram)
    bigram = prev_uni + unigram
    #if prev_node and isContentWord(prev_node):
    if prev_node and not isSymbol(prev_node) and not isSymbol(node):
      ngrams.append(bigram)
    if isContentWord(node) and not isStopWord(base) and len(base) > 1:
      ngrams.append(base)
    preb_tri = trigram
    prev_bi = bigram
    prev_uni = unigram
    prev3_node = prev2_node
    prev2_node = prev_node
    prev_node = node
    node = node.next
  return ngrams

def toDataFrame(featvecs, all_feats):
  dict = {}
  for feat in all_feats:
    values = []
    for featvec in featvecs:
      if feat in featvec.keys():
        values.append(featvec[feat])
      else:
        values.append(0)
    dict[feat] = values
  return pd.DataFrame.from_dict(dict)

def corr_spearman(a, b, alpha = 0.95):
  r = scipy.stats.spearmanr(a, b)[0]
  n = len(a)
  if n <= 3:
      AssertionError("Not enough amount data")
  z= 0.5 * numpy.log((1+r)/(1-r))
  za = scipy.stats.norm.ppf(0.5 + 0.5 * alpha) 
  zl = z - za * math.sqrt(1/(n-3))
  zu = z + za * math.sqrt(1/(n-3))
  rhol = (math.exp(2 * zl) - 1 )/ (math.exp(2 * zl) +1 )
  rhou = (math.exp(2 * zu) - 1 )/ (math.exp(2 * zu) +1 )
  return round(rhol, 3), round(r, 3), round(rhou, 3)

stopwords = ["する", "なる", "ある", "いる", "やる", "いう", "おく", "ない", "しまう", "もの", "せる",
             "とき", "こと", "ため", "ほう", "ところ", "よる", "とも", "した", "のか", "の", "だ"]

thresh_freq = 3

def ext_feature(xlsx_file, cols, cols_multi, quant_rule, cols_rawquant, cols_text):
  df = pd.read_excel(xlsx_file)
  featvecs = []
  feat_freq = {}
  for index, row in df.iterrows():
    featvec = {}
    text = ""
    #print(row)
    for col in cols:
      if col in cols_multi:
        feats = row[col].split(";")
        for feat in feats:
          if len(feat) == 0: 
            continue
          feat += "（" + col + "）"
          #print("@@@ " + feat)
          featvec[feat] = 1
          inc(feat_freq, feat)
      else: 
        feat = row[col] + "（" + col + "）"
        #print(feat)
        featvec[feat] = 1
        inc(feat_freq, feat)
    for col in quant_rule.keys():
      feat = "（" + col + "）"
      inc(feat_freq, feat)
      if row[col] in quant_rule[col].keys():
        featvec[feat] = quant_rule[col][row[col]]
      else:
        featvec[feat] = 0
    for col in cols_rawquant:
      feat = "（" + col + "）"
      featvec[feat] = row[col]
      inc(feat_freq, feat)
    for col in cols_text:
      if type(row[col]) == float:
        continue
      text += col+"「"+row[col]+"」"
      sentences = row[col].split("。")
      for sentence in sentences:
        ngrams = get_ngrams(sentence)
        for ngram in ngrams:
          if len(ngram) > 0:
            feat = "「"+ ngram + "」（" + col + "）" 
            featvec[feat] = 1
            inc(feat_freq, feat)
    if len(text) > 0:
      featvec["text"] = text
    featvecs.append(featvec)
  # 閾値未満の頻度の特徴を削除
  feat_del = []
  all_feats = list(feat_freq.keys())
  for feat in feat_freq.keys():
    if (feat_freq[feat] < thresh_freq):
      feat_del.append(feat)
      all_feats.remove(feat)
  for featvec in featvecs:
    for feat in feat_del:
      if feat in featvec.keys():
        del featvec[feat]
  return (featvecs, all_feats)

def get_expl_features(expls, all_feats):
  expl_feats = []
  for feat in all_feats:
    for expl in expls:
      if feat.find("（"+expl+"）") >= 0:
        expl_feats.append(feat)
  return expl_feats

thresh_r0 = 0.1

def analysis(df, expl_feats, all_feats, quant_rule):
  result = {}
  rev_rule = inverse_rule(quant_rule)
  #print(rev_rule)
  for expl_feat in expl_feats:
    result[expl_feat] = {"posi": {}, "nega": {}}
    s1 = df[expl_feat]
    for feat in all_feats:
      if feat != expl_feat:
        s2 = df[feat]
        (rl, r, ru) = corr_spearman(s1, s2)
        if r > 0 and rl > thresh_r0:
          result[expl_feat]["posi"][feat] = {}
          result[expl_feat]["posi"][feat]["r"] = r
          result[expl_feat]["posi"][feat]["rl"] = rl
          result[expl_feat]["posi"][feat]["ru"] = ru
          result[expl_feat]["posi"][feat]["r0"] = rl
        elif r < 0 and ru < -thresh_r0:
          result[expl_feat]["nega"][feat] = {}
          result[expl_feat]["nega"][feat]["r"] = r
          result[expl_feat]["nega"][feat]["rl"] = rl
          result[expl_feat]["nega"][feat]["ru"] = ru
          result[expl_feat]["nega"][feat]["r0"] = abs(ru)
  freq = {}
  for expl_feat in expl_feats:
    # 頻度を数える
    s1 = df[expl_feat]
    freq[expl_feat] = {}
    for val in s1:
      feat = restore_feat(val, expl_feat, rev_rule)  
      inc(freq[expl_feat], feat)
  for expl_feat in expl_feats:
    labels = []
    freq_vals = []
    sorted_item = []
    if expl_feat.startswith("（"):
      key = expl_feat[1:len(expl_feat)-1]
      sorted_item = sorted(freq[expl_feat].keys(), key=lambda x: quant_rule[key][x] if x in quant_rule[key].keys() else 0, reverse=True)
    else:
      feat = "選択せず"
      if (val > 0):
        feat = expl_feat[0:expl_feat.index("（")]
      sorted_item = sorted(freq[expl_feat].keys(), key=lambda x: quant_rule[key][x] if x in quant_rule[key].keys() else 0, reverse=True)
    for feat in sorted_item:
      labels.append(feat)
      freq_vals.append(freq[expl_feat][feat])
    result[expl_feat]["freq"] = {"labels": labels, "data": freq_vals}
  for expl_feat in expl_feats:
    # ソート
    sorted_posi = sorted(result[expl_feat]["posi"], key=lambda x: result[expl_feat]["posi"][x]["r0"], reverse=True)
    result[expl_feat]["posi"] = sorted_feats(result[expl_feat]["posi"], sorted_posi)
    sorted_nega = sorted(result[expl_feat]["nega"], key=lambda x: result[expl_feat]["nega"][x]["r0"], reverse=True)
    result[expl_feat]["nega"] = sorted_feats(result[expl_feat]["nega"], sorted_nega)   
  return result

def restore_feat(val, expl_feat, rev_rule):
  if expl_feat.startswith("（"):
    feat = "その他"
    key = expl_feat[1:len(expl_feat)-1]
    if val in rev_rule[key].keys():
      feat = rev_rule[key][val]
    return feat
  else:
    feat = "選択せず"
    if (val > 0):
      feat = expl_feat[0:expl_feat.find("（")]
    return feat

def sorted_feats(dict, sorted_key):
  sorted = collections.OrderedDict()
  for key in sorted_key:
    sorted[key] = dict[key]
  return sorted

def write_result_json(result, json_file):
  out = open(json_file, 'w')
  out.write(json.dumps(result))
  out.close()

def emphasize(text, feat):
  if feat.find("「") == 0 and feat.find("」") > 0:
    ngram = feat[1:feat.index("」")]
    if text.find(ngram) < 0:
      ngram = ngram[0:len(ngram)-1]
    return text.replace(ngram, "<strong>"+ngram+"</strong>")
  else:
    return text

def regist_texts(featvecs, result):
  texts = {}
  for expl_feat in result.keys():
    texts[expl_feat+"-posi"] = {}
    texts[expl_feat+"-nega"] = {}
    for feat in result[expl_feat]["posi"]:
      texts[expl_feat+"-posi"][feat] = []
    for feat in result[expl_feat]["nega"]:
      texts[expl_feat+"-nega"][feat] = []
  for featvec in featvecs:
    if not "text" in featvec.keys():
      continue
    text = featvec["text"]
    for feat in featvec.keys():
      emp_text = emphasize(text, feat)
      for expl_feat in result.keys():
        if expl_feat in featvec.keys() and featvec[expl_feat] > 0 and feat in result[expl_feat]["posi"].keys():
          texts[expl_feat+"-posi"][feat].append({"text": emp_text, "val": featvec[expl_feat]})
        elif (not (expl_feat in featvec.keys()) or featvec[expl_feat] < 0) and feat in result[expl_feat]["nega"]:
          texts[expl_feat+"-nega"][feat].append({"text": emp_text, "val": featvec[expl_feat] if expl_feat in featvec.keys() else 0})
  for expl_feat in result.keys():
    for feat in result[expl_feat]["posi"].keys():
      texts[expl_feat+"-posi"][feat].sort(key=lambda x: sim(expl_feat + feat, x["text"]), reverse=True)
    for feat in result[expl_feat]["nega"].keys():
      texts[expl_feat+"-nega"][feat].sort(key=lambda x: sim(expl_feat + feat, x["text"]), reverse=True)
  return texts

def inverse_rule(quant_rule):
  rev = {}
  for col in quant_rule:
    dic = quant_rule[col]
    rev[col] = {}
    for feat in quant_rule[col]:
      rev[col][quant_rule[col][feat]] = feat
  return rev

In [ ]:
cols = ["所属課程", "学年", "受講場所", "使用機器", "通信環境", "技術的問題", "良かったこと", "悪かったこと", "動画教材への要望"]
cols_multi = ["使用機器", "通信環境", "良かったこと", "悪かったこと", "動画教材への要望"]
quant_rule = {"対面授業の代替":{"できていた":2, "ややできていた":1, "あまりできていなかった":-1, "できていなかった":-2},
              "授業理解度":{"よく理解できた":2, "まあまあ理解できた":1, "あまり理解できなかった":-1, "理解できなかった":-2},
              "教員とのコミュニケーション":{"頻繁に質問した":2, "時々質問した":1, "全く質問しなかった":-1},
              "全動画の視聴":{"履修登録した全ての動画を視聴した":1}}
cols_rawquant = ["複数回視聴割合"]
cols_text = ["技術的問題の内容", "教員が映って欲しい理由", "今後の遠隔授業の活用方法", "感想など"]

(featvecs_stu, all_feats) = ext_feature("/content/drive/My Drive/enq_analysis/data/student.xlsx", cols, cols_multi, quant_rule, cols_rawquant, cols_text)
df_stu = toDataFrame(featvecs_stu, all_feats)
expl_feats = [
 '（対面授業の代替）', '（授業理解度）', '（教員とのコミュニケーション）', 
 '自分のペースで学習できる（良かったこと）',
 '自宅で学習できる（良かったこと）',
 '教室より集中できる（良かったこと）',
 '復習ができる（良かったこと）',
 '教材がわかりやすい（良かったこと）',
 '先生に質問がしやすい（良かったこと）',
 'コンピュータやオンラインツールについて知識やスキルが高まる（良かったこと）',
 '特にない（良かったこと）',
 'コンピュータの操作に慣れていない（悪かったこと）',
 '集中力が続かない（悪かったこと）',
 '孤立感を感じる（悪かったこと）',
 '課題が多い（悪かったこと）',
 'Moodleの使い方がわからない（悪かったこと）',
 'インターネット環境が十分でない（悪かったこと）',
 'PC, プリンターの機器面（悪かったこと）',
 '画像・音声の品質が悪く, 授業の教材がわかりにくい（悪かったこと）',
 '先生に質問がしにくい（悪かったこと）',
 '勉強のペースがつかみにくい（悪かったこと）']
#expls = ["良かったこと", "悪かったこと", "授業理解度", "対面授業の代替", "教員とのコミュニケーション", "複数視聴割合"]
#expl_feats = get_expl_features(expls, all_feats)

result = analysis(df_stu, expl_feats, all_feats, quant_rule)
write_result_json(result, "/content/drive/My Drive/enq_analysis/data/student.json")
texts = regist_texts(featvecs_stu, result)
for label in texts.keys():
  write_result_json(texts[label], "/content/drive/My Drive/enq_analysis/data/student-"+label+".json") 
result




In [ ]:
cols = ["所属課程", "学年", "受講場所", "使用機器", "通信環境", "技術的問題", "良かったこと", "悪かったこと", "動画教材への要望"]
cols_multi = ["使用機器", "通信環境", "良かったこと", "悪かったこと", "動画教材への要望"]
quant_rule = {"対面授業の代替":{"できていた":2, "ややできていた":1, "あまりできていなかった":-1, "できていなかった":-2},
              "授業理解度":{"よく理解できた":2, "まあまあ理解できた":1, "あまり理解できなかった":-1, "理解できなかった":-2},
              "教員とのコミュニケーション":{"頻繁に質問した":2, "時々質問した":1, "全く質問しなかった":-1},
              "全動画の視聴":{"履修登録した全ての動画を視聴した":1}}
cols_rawquant = ["複数視聴割合"]
cols_text = ["技術的問題の内容", "教員が映って欲しい理由", "今後の遠隔授業の活用方法", "感想など"]

(featvecs_stu, all_feats) = ext_feature("/content/drive/My Drive/enq_analysis/data/student.xlsx", cols, cols_multi, quant_rule, cols_rawquant, cols_text)
text_stu = regist_texts(featvecs_stu)
df_stu = toDataFrame(featvecs_stu, all_feats)

#expls = ["良かったこと", "悪かったこと", "授業理解度", "対面授業の代替", "教員とのコミュニケーション", "複数視聴割合"]
#expl_feats = get_expl_features(expls, all_feats)
expl_feats = [
 '（対面授業の代替）', '（授業理解度）', '（教員とのコミュニケーション）', '（複数視聴割合）',
 '自分のペースで学習できる（良かったこと）',
 '自宅で学習できる（良かったこと）',
 '教室より集中できる（良かったこと）',
 '復習ができる（良かったこと）',
 '教材がわかりやすい（良かったこと）',
 '先生に質問がしやすい（良かったこと）',
 'コンピュータやオンラインツールについて知識やスキルが高まる（良かったこと）',
 '特にない（良かったこと）',
 'コンピュータの操作に慣れていない（悪かったこと）',
 '集中力が続かない（悪かったこと）',
 '孤立感を感じる（悪かったこと）',
 '課題が多い（悪かったこと）',
 'Moodleの使い方がわからない（悪かったこと）',
 'インターネット環境が十分でない（悪かったこと）',
 'PC, プリンターの機器面（悪かったこと）',
 '画像・音声の品質が悪く, 授業の教材がわかりにくい（悪かったこと）',
 '先生に質問がしにくい（悪かったこと）',
 '勉強のペースがつかみにくい（悪かったこと）']

result = analysis(df_stu, expl_feats, all_feats)
write_result_json(result, "/content/drive/My Drive/enq_analysis/data/student.json")
result


In [ ]:
!git config --global user.email "siramatu@gmail.com"
!git config --global user.name "Shun Shiramatsu"

